In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [4]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.5 MB/s 
     |████████████████████████████████| 210 kB 48.8 MB/s 
     |████████████████████████████████| 81 kB 7.7 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 146 kB 55.7 MB/s 
     |████████████████████████████████| 113 kB 59.8 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=98121b5e3fa31f1f89f41e41af9274f16ebd728eca62feef0635a0ab766e2bdc
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [5]:
import optuna

In [6]:
train = pd.read_csv('/content/drive/MyDrive/project9/train_transformed.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/project9/music30s_trainlabel.csv')
test =pd.read_csv('/content/drive/MyDrive/project9/test_transformed.csv')


In [7]:
train_labels =train_labels.iloc[:,1:]
train_labels.head(3)


,label
0,reggae
1,reggae
2,country


In [8]:
cols = [c for c in train.columns]

In [9]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
train[cols] = stdsc.fit_transform(train[cols])
test[cols] = stdsc.transform(test[cols])

In [10]:
display(train.head(3))
display(test.head(3))

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.211645,0.314663,-0.234117,...,1.838597,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-0.925636,-1.493513,0.877118,...,0.824710,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.213437,0.450461,-0.695707,...,0.631436,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858


,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,-0.648987,-1.134700,-1.172775,-0.967173,-1.282262,-0.778708,-1.049366,-0.701822,0.365950,...,1.438900,1.531189,1.933873,1.593768,1.864126,0.803889,1.817925,1.818024,1.261632,1.923743
1,-0.142344,0.703835,-1.919805,-1.723546,-1.199076,-1.699029,-1.720368,0.207379,0.356319,0.877118,...,-1.153414,-1.881063,-3.674341,-3.609437,-3.750683,-3.791344,-3.700374,-3.402824,-2.418003,-1.934408
2,-0.298576,0.939176,2.349185,0.841052,0.373789,0.601497,-0.148723,0.212788,0.200619,-0.948482,...,1.184065,0.593270,1.220743,0.620820,0.712082,0.518463,0.468916,0.123271,-0.013371,0.258653


In [11]:
train = pd.concat([train,train_labels],axis=1)

In [12]:
N_SPLITS = 5
N_REPEATS = 3
EARLY_STOPPING_ROUNDS = 300
VERBOSE = False
SEED = 17171

In [13]:
target_column = 'label'

In [14]:
features = [c for c in train.columns if c not in [target_column]]

In [15]:
X = train.drop('label',axis=1)
y=train['label']

In [16]:
## going by the optuna way:
import optuna
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
# Which hyperparameters to tune: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

def objective(trial):

    # -- Tune estimator algorithm
    #suggest_int(name, low, high[, step, log])
    C = trial.suggest_float("C", 0,100)
    #algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
    gamma = trial.suggest_categorical("gamma",['auto', 'scale'])
    kernel = trial.suggest_categorical("kernel", ['linear', 'poly', 'rbf','sigmoid'])
    svc = SVC(C=C, gamma=gamma, kernel=kernel)
        
    # -- Cross-validate the features reduced by dimensionality reduction methods
    rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
    score = cross_val_score(svc, X,y, scoring='accuracy', cv=rskfold, n_jobs = -1, error_score = 'raise')
    score = score.mean()
    return score


In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout = 4*60*60)

[I 2022-04-24 09:37:51,069] A new study created in memory with name: no-name-71b36170-da0f-444d-919f-b6c1a93a65cb
[I 2022-04-24 09:37:53,835] Trial 0 finished with value: 0.7559259259259259 and parameters: {'C': 96.51726116137355, 'gamma': 'auto', 'kernel': 'rbf'}. Best is trial 0 with value: 0.7559259259259259.
[I 2022-04-24 09:37:54,449] Trial 1 finished with value: 0.5111111111111112 and parameters: {'C': 81.81466852197474, 'gamma': 'scale', 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.7559259259259259.
[I 2022-04-24 09:37:55,559] Trial 2 finished with value: 0.7529629629629631 and parameters: {'C': 99.33487793918557, 'gamma': 'scale', 'kernel': 'rbf'}. Best is trial 0 with value: 0.7559259259259259.
[I 2022-04-24 09:37:56,407] Trial 3 finished with value: 0.6996296296296297 and parameters: {'C': 25.38112464118667, 'gamma': 'auto', 'kernel': 'linear'}. Best is trial 0 with value: 0.7559259259259259.
[I 2022-04-24 09:37:57,135] Trial 4 finished with value: 0.6640740740740741 a

In [18]:
trial = study.best_trial    
print('Accuracy: {}'.format(trial.value))   

Accuracy: 0.7648148148148147


In [19]:
print("Best hyperparameters: {}".format(trial.params))

Best hyperparameters: {'C': 6.836810780580301, 'gamma': 'auto', 'kernel': 'rbf'}


In [20]:
optuna.visualization.plot_optimization_history(study)


In [21]:
svc_params = trial.params

In [22]:
svc_params = {'C': 69.02071553922178, 'gamma': 'scale', 'kernel': 'rbf'}

{'C': 6.836810780580301, 'gamma': 'auto', 'kernel': 'rbf'}

In [33]:

from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import classification_report
import statistics
from statistics import mean

In [39]:
%%time
N_SPLITS = 5
y_preds = []
acc= []
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, valid_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[valid_id]
  y_valid = y.iloc[valid_id]
  model = SVC(**svc_params)
  
  model.fit(X_train, y_train)
  valid_pred = model.predict(X_valid)

  #print('fold:{}'.format(fold+1))
  #print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)
  acc.append(accuracy_score(y_valid, valid_pred))

  y_preds.append(model.predict(test))
print('the mean accuracy is: {}'.format(mean(acc)))




the mean accuracy is: 0.76
CPU times: user 445 ms, sys: 881 µs, total: 446 ms
Wall time: 446 ms


In [40]:

y_preds

[array(['country', 'jazz', 'hiphop', 'reggae', 'blues', 'country', 'pop',
        'jazz', 'blues', 'blues', 'disco', 'disco', 'metal', 'reggae',
        'jazz', 'country', 'country', 'reggae', 'jazz', 'blues', 'country',
        'hiphop', 'pop', 'classical', 'disco', 'reggae', 'disco', 'metal',
        'country', 'jazz', 'jazz', 'classical', 'rock', 'classical',
        'rock', 'rock', 'classical', 'classical', 'pop', 'blues', 'rock',
        'blues', 'jazz', 'classical', 'blues', 'blues', 'jazz', 'blues',
        'pop', 'pop', 'disco', 'blues', 'hiphop', 'rock', 'jazz', 'pop',
        'blues', 'blues', 'disco', 'reggae', 'blues', 'pop', 'pop',
        'country', 'classical', 'rock', 'jazz', 'reggae', 'reggae',
        'metal', 'reggae', 'pop', 'metal', 'classical', 'classical',
        'jazz', 'hiphop', 'blues', 'disco', 'reggae', 'metal', 'country',
        'hiphop', 'blues', 'classical', 'blues', 'blues', 'disco', 'disco',
        'metal', 'country', 'blues', 'metal', 'metal', 'blue

In [25]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None




In [26]:
y_pred = pd.DataFrame(columns=['label'])
pred= []
for j in range(0,100):
    arr = [y_preds[0][j], y_preds[1][j], y_preds[2][j], y_preds[3][j],y_preds[4][j]]
    pred.append(mode(arr))

In [27]:
y_pred["label"]=pred

In [28]:
y_pred

,label
0,country
1,jazz
2,hiphop
3,reggae
4,blues
...,...
95,rock
96,disco
97,pop
98,pop


In [29]:
y_pred.to_csv('svc_optuna_pred.csv', index=False)

In [30]:
!cp -r '/content/svc_optuna_pred.csv' /content/drive/MyDrive/project9
